In [1]:
import tensorflow.compat.v1 as tf
import cv2
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.__version__

'2.1.0-dev20191123'

In [3]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6996200370140416180
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4324876248875108227
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6796217413183888674
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5633736704
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14870933296649624740
physical_device_desc: "device: 0, name: GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [8]:
a = tf.truncated_normal([16,128,128,3])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.shape(a))

array([ 16, 128, 128,   3], dtype=int32)

In [9]:
b = tf.reshape(a, [16, 49152])
sess.run(tf.shape(b))

array([   16, 49152], dtype=int32)

# Actual Code ;)

In [10]:
import dataset
import time, math, random, os
import numpy as np

from datetime import timedelta

In [11]:
# Set the random seed as constant for consistent results
from numpy.random import seed
seed(3)
tf.set_random_seed(2)

In [12]:
batch_size = 32

In [13]:
# Prepare the input data
classes = os.listdir('images')
num_classes = len(classes)

print((classes, num_classes))

(['NSFW', 'SFW'], 2)


In [14]:
# 20% data will be used for validation
validation_size = 0.2
img_size = 128
num_channels = 3
train_path = 'images'

In [15]:
# Loading the training and validation data set and labels into memory and use that during training
data = dataset.read_train_sets(train_path=train_path, image_size=img_size, classes=classes, validation_size=validation_size)

Going to read training images
Now going to read NSFW files (Index: 0)
Now going to read SFW files (Index: 1)


In [16]:
print("Complete reading input data. Will Now print a snippet of it")
print("Number of files in Training-set:\t{}".format(len(data.train.labels)))
print("Number of files in Validation-set:\t{}".format(len(data.valid.labels)))

Complete reading input data. Will Now print a snippet of it
Number of files in Training-set:	1345
Number of files in Validation-set:	336


In [17]:
session = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, img_size, img_size, num_channels], name='x')

In [18]:
# Labels
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

Instructions for updating:
Use the `axis` argument instead


In [19]:
# Network graph params
filter_size_conv1 = 3
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 32

filter_size_conv3 = 3
num_filters_conv3 = 64

fc_layer_size = 128

In [20]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

In [21]:
def create_convolutional_layer(input,
                              num_input_channels,
                              conv_filter_size,
                              num_filters):
    ## Weights that will be trained using create weights function
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## Biases to train
    biases = create_biases(num_filters)
    
    ## creating the convolution layer
    layer = tf.nn.conv2d(input=input,
                        filter= weights,
                        strides=[1,1,1,1],
                        padding = 'SAME')
    layer += biases
    
    ## Using max pooling
    layer = tf.nn.max_pool(value=layer,
                          ksize=[1,2,2,1],
                          strides=[1,2,2,1],
                          padding = 'SAME')
    
    ## Feeding output of pooling to ReLu which is the activation function
    layer = tf.nn.relu(layer)
    
    return layer

In [22]:
def create_flatten_layer(layer):
    # We know that the shape of the layer will be [batch_size img_size img_size num_channels] 
    # But let's get it from the previous layer.
    
    layer_shape = layer.get_shape()
    
    ## Number of features will be img_height * img_width* num_channels. But we shall calculate it in place of hard-coding it.
    num_features = layer_shape[1:4].num_elements()
    
    ## Flattening the layer to reshape to num of features
    layer = tf.reshape(layer, [-1, num_features])
    
    return layer

In [23]:
def create_fc_layer(input,
                   num_inputs,
                   num_outputs,
                   use_relu = True):
    
    ## Defining trainable weights and biases
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
    
    ## FC layer takes x and gives us wx + b which are matrices so we use tf.matmul
    layer = tf.matmul(input, weights) + biases
    
    if use_relu:
        layer = tf.nn.relu(layer)
    
    return layer

In [24]:
layer_conv1 = create_convolutional_layer(input=x,
                                         num_input_channels=num_channels,
                                         conv_filter_size=filter_size_conv1,
                                         num_filters=num_filters_conv1)

layer_conv2 = create_convolutional_layer(input=layer_conv1, 
                                         num_input_channels=num_filters_conv1, 
                                         conv_filter_size=filter_size_conv2,
                                         num_filters=num_filters_conv2)

layer_conv3 = create_convolutional_layer(input=layer_conv2, 
                                         num_input_channels=num_filters_conv2, 
                                         conv_filter_size=filter_size_conv3,
                                         num_filters=num_filters_conv3)

layer_flat = create_flatten_layer(layer=layer_conv3)

layer_fc1 = create_fc_layer(input=layer_flat, 
                            num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                            num_outputs=fc_layer_size,
                            use_relu=True)

layer_fc2 = create_fc_layer(input=layer_fc1, 
                            num_inputs=fc_layer_size, 
                            num_outputs= num_classes, 
                            use_relu=False) 

In [25]:
type(layer_fc2)

tensorflow.python.framework.ops.Tensor

In [26]:
y_pred = tf.nn.softmax(layer_fc2, name='y_pred')
y_pred_cls = tf.argmax(y_pred, dimension=1)
session.run(tf.global_variables_initializer())
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


session.run(tf.global_variables_initializer())

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [27]:
def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

In [28]:
saver = tf.train.Saver()

In [29]:
total_iterations = 0
def train(num_iteration):
    global total_iterations
    tf.debugging.set_log_device_placement(True)
    
    for i in range(total_iterations, total_iterations + num_iteration):
        ## print("Iteration number: {0}".format(i))
        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size=batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(batch_size=batch_size)
        
        feed_dict_tr = {x: x_batch, y_true:y_true_batch}
        feed_dict_val = {x: x_valid_batch, y_true: y_valid_batch}
        
        session.run(optimizer, feed_dict=feed_dict_tr)
        
        if i%int(data.train.num_examples / batch_size) == 0:
            val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(data.train.num_examples / batch_size))
            
            show_progress(epoch=epoch, feed_dict_train=feed_dict_tr, feed_dict_validate=feed_dict_val, val_loss=val_loss)
            
            saver.save(sess=session, save_path='./trained-model/nsfw-model')
            
    total_iterations += num_iteration



In [30]:
train(num_iteration=3000)

Training Epoch 1 --- Training Accuracy:  62.5%, Validation Accuracy:  71.9%,  Validation Loss: 0.634
Training Epoch 2 --- Training Accuracy:  62.5%, Validation Accuracy:  50.0%,  Validation Loss: 0.686
Training Epoch 3 --- Training Accuracy:  62.5%, Validation Accuracy:  65.6%,  Validation Loss: 0.607
Training Epoch 4 --- Training Accuracy:  62.5%, Validation Accuracy:  84.4%,  Validation Loss: 0.456
Training Epoch 5 --- Training Accuracy:  62.5%, Validation Accuracy:  75.0%,  Validation Loss: 0.535
Training Epoch 6 --- Training Accuracy:  68.8%, Validation Accuracy:  71.9%,  Validation Loss: 0.503
Training Epoch 7 --- Training Accuracy:  71.9%, Validation Accuracy:  75.0%,  Validation Loss: 0.549
Training Epoch 8 --- Training Accuracy:  71.9%, Validation Accuracy:  68.8%,  Validation Loss: 0.598
Training Epoch 9 --- Training Accuracy:  71.9%, Validation Accuracy:  87.5%,  Validation Loss: 0.372
Training Epoch 10 --- Training Accuracy:  71.9%, Validation Accuracy:  78.1%,  Validation L

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [6]:
import numpy as np

In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
tf.__version__

In [11]:
def predict(file_name, meta_graph, checkpoint):
    image_size = 128
    num_channels = 3
    images = []
    
    image = cv2.imread(file_name)
    
    image = cv2.resize(image, (image_size, image_size), 0,0, cv2.INTER_LINEAR)
    
    images.append(image)
    images = np.array(images, dtype=np.uint8)
    
    images = images.astype('float32')
    images = np.multiply(images, 1.0 / 255.0)
    
    x_batch = images.reshape(1, image_size, image_size, num_channels)
    
    sess = tf.Session()
    
    saver = tf.train.import_meta_graph(meta_graph)
    
    saver.restore(sess,save_path=tf.train.latest_checkpoint(checkpoint))
    
    graph = tf.get_default_graph()
    
    # Now, let's get hold of the op that we can be processed to get the output.
    # In the original network y_pred is the tensor that is the prediction of the network
    y_pred = graph.get_tensor_by_name("y_pred:0")
    
    ## Let's feed the images to the input placeholders
    x= graph.get_tensor_by_name("x:0") 
    #y_true = graph.get_tensor_by_name("y_true:0") 
    #y_test_images = np.zeros((1, len(os.listdir('training_data'))))
    
    feed_dict_testing = {x: x_batch}
    result=sess.run(y_pred, feed_dict=feed_dict_testing)
    
    print(result)
    
    return result

In [24]:
predictions = predict("./images/D7YZJFh.jpg", "./trained-model/nsfw-model.meta", "./trained-model/")
np.argmax(predictions[0])

INFO:tensorflow:Restoring parameters from ./trained-model/nsfw-model
[[0.9989889  0.00101107]]


0

In [16]:
np.argmax(predictions[0])

0